[![Generic badge](https://img.shields.io/badge/License-Properitary-red.svg)](https://github.com/JanGaida/research_project_machine_learning_hshof_sose2020/blob/master/LICENSE.md) <br>
[![Active Development](https://img.shields.io/badge/Maintenance%20Level-Actively%20Developed-brightgreen.svg)](https://github.com/JanGaida/research_project_machine_learning_hshof_sose2020/) <br>
[![Ask Me Anything !](https://img.shields.io/badge/Ask%20me-anything-1abc9c.svg)](https://github.com/JanGaida/research_project_machine_learning_hshof_sose2020/issues) <br>
[![Python 3.7.7](https://img.shields.io/badge/python-3.7.7-blue.svg)](https://www.python.org/downloads/release/python-377/)

# <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/50/Logo_fh_hof.svg/2000px-Logo_fh_hof.svg.png" width="300" style="background-color:#FFF">

<font size="+4"><i><u>Q-Learning mit 'Super Mario Bros'</u></i></font>

Seminararbeit der Vorlesung **Angewandtes Maschinelles Lernen** an der **Hochschule für angewande Wissenschaften Hof** des **Sommersemesters 2020**.

# Terminologie

<a href="http://www.incompleteideas.net/book/ebook/node28.html"><img src="http://www.incompleteideas.net/book/ebook/figtmp7.png"/></a>

- <u><b>Agent:</b></u> Der Lernern und Entscheidungsfinder.
- <u><b>Environment:</b></u> Dort wo der Agent lernt und entscheidet welche Aktionen er durchführt.
- <u><b>Action:</b></u> Ein Set von Aktionen die der Agent durchführen kann.
- <u><b>State:</b></u> Der Status des Agenten im Enviorment.
- <u><b>Reward:</b></u> Für jede Action ausgewählt durch den Agenten berrechnet das Enviorment einen Reward.
- <u><b>Policy:</b></u> Die Entscheidungsfindendungs-Funktion (vgl. 'control strategy') des Agenten, welcher eine Set von Situationen zu Aktionen representiert.
- <u><b>Value Funktion:</b></u> Eine Set von State's zu realen Zahlen, wo der Wert eines State den longterm Reward ausgehend vom Start des States representiert.
- <u><b>Function approximator:</b></u> Bezieht sich auf das Problem des Induzierens einer Funktion aus Trainingsbeispielen. Standard-Approximatoren umfassen Entscheidungsbäume, neuronale Netze und Nähster-Nachbar-Methoden.
- <u><b>Markov decision process (MDP):</b></u> Ein Wahrscheinlichkeitsmodell eines sequentiellen Entscheidungsproblems, bei dem Zustände genau wahrgenommen werden können und der aktuelle Zustand und die ausgewählte Aktion eine Wahrscheinlichkeitsverteilung auf zukünftige Zustände bestimmen. Im Wesentlichen hängt das Ergebnis der Anwendung einer Aktion auf einen Status nur von der aktuellen Aktion und dem aktuellen Status ab (und nicht von vorhergehenden Aktionen oder Status).
- <u><b>Dynamic programming (DP):</b></u> Ist eine Klasse von Lösungsmethoden zur Lösung sequentieller Entscheidungsprobleme mit einer kompositorischen Kostenstruktur. Richard Bellman war einer der Hauptgründer dieses Ansatzes.
- <u><b>Monte Carlo methods:</b></u> Eine Klasse von Methoden zum Erlernen von Wertfunktionen, die den Wert eines Zustands durch Ausführen vieler Versuche ab diesem Zustand schätzt und dann die Gesamtbelohnungen für diese Versuche mittelt.
- <u><b>Temporal Difference (TD) algorithms:</b></u> Eine Klasse von Lernmethoden, die auf der Idee basiert, zeitlich aufeinanderfolgende Vorhersagen zu vergleichen. Möglicherweise die grundlegendste Idee beim gesamten Reinforcement Learning.
- <u><b>Model:</b></u> Die Ansicht des Agent über das Enviorment, in der Zustands-Aktions-Paare Wahrscheinlichkeitsverteilungen über Zustände zugeordnet werden. Beachten Sie, dass nicht jeder Reinforcement-Learning-Agent ein Modell seiner Umgebung verwendet muss.

# Vorbereitungen

*Code der das Jupyter-Notebook vorbereit.*

## Notebook-Styling

*Ändert u.a. das Layout des Notebooks.*

In [1]:
from IPython.core.display import display, HTML

_notebook_relative_width = "80%" # Wertebereich: 0-100, muss nach HTML-Schema definiert sein
display(HTML('<style>.container { width: ' + _notebook_relative_width + ' !important; }</style>'))

## Requirements

*Die benötigten Packages wurden aus der bei mir installierten Pip-Liste gebildet.*

**Hinweis:** *Es werden* ***mehr als zwingend notwendige*** *Packages installiert.*

In [2]:
# Package-Liste
#!pip list

In [3]:
%%capture
# Installation
#!pip install -r requirements.txt

In [4]:
%%capture
# Update 
#!pip list --outdated --format=freeze | grep -v '^\-e' | cut -d = -f 1  | xargs -n1 pip install -U

In [5]:
%%capture
# Export Pip-Packages nach './requirements.txt'
!pip freeze > requirements.txt

# Konfiguration

*Import-Statements & Vorbereitung der Frameworks/Module zusammengefasst.*

## Allgemein

*Nicht weiter spezifizierte Import-Statements, bspw. von System-Bibliotheken.*

In [6]:
import os, glob, uuid, shutil

import time
from datetime import datetime

import numpy as np

import random

from collections import deque

In [7]:
from IPython.core.display import display, HTML

# Rendert den übergebenen Content als HTML-Objekt
def html(content):
    display(HTML(content))
    
# Zeigt einen Paragraphen mit der angegeben Farbe (default: Schwarz)
def html_p(content, color = 'black', bold = False):
    if bold:
        html('<p style="color:{};"><b>{}</b></p>'.format(color, content))
    else:
        html('<p style="color:{};">{}</p>'.format(color, content))

## Ordner

### Logs

In [8]:
logs_base_dir = './logs'

if os.path.isdir(logs_base_dir):
    for f in glob.glob('{}/*'.format(logs_base_dir)):
        if os.path.isdir(f):
            for f2 in glob.glob('{}/*'.format(f)):
                html_p('\'{}\' gelöscht.'.format(f2),'red',True)
                os.remove(f2)
            html_p('\'{}\'-Ordner gelöscht.'.format(f),'red',True)
            shutil.rmtree(f)
        else:
            html_p('\'{}\' gelöscht.'.format(f),'red',True)
            os.remove(f)
            
    html_p('\'{}\'-Ordner geleert.'.format(logs_base_dir),'gray',True)
else:
    os.mkdir(logs_base_dir)
    html_p('\'{}\'-Ordner erstellt.'.format(logs_base_dir),'gray',True)

### Records

In [9]:
rec_base_dir = './records'

if not os.path.isdir(rec_base_dir):
    os.mkdir(rec_base_dir)
    html_p('\'{}\'-Ordner erstellt.'.format(rec_base_dir),'gray',True)
else:
    html_p('\'{}\'-Ordner bereits vorhanden.'.format(rec_base_dir),'green',True)

### Models

In [10]:
model_base_dir = './models'

if not os.path.isdir(model_base_dir):
    os.mkdir(model_base_dir)
    html_p('\'{}\'-Ordner erstellt.'.format(model_base_dir),'gray',True)
else:
    html_p('\'{}\'-Ordner bereits vorhanden.'.format(model_base_dir),'green',True)

## Tensorflow

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/11/TensorFlowLogo.svg/1280px-TensorFlowLogo.svg.png" width="150px"/>

*TensorFlow ist ein Framework zur datenstromorientierten Programmierung. Es wird aus Python-Programmen heraus benutzt und ist in Python und C++ implementiert. Populäre Anwendung findet TensorFlow im Bereich des maschinellen Lernens. [...]*

[Weiter Informationen](https://pypi.org/project/tensorflow/)

In [11]:
%%capture
import tensorflow as tf

## Tensorboard

<img src="https://www.tensorflow.org/site-assets/images/project-logos/tensorboard-logo-social.png" width="250px"/>

*TensorBoard ist ein Sammlung von Web-Applicationen um TensorFlow zu inspizieren und zu verstehen.*

[Weitere Informationen](https://pypi.org/project/tensorboard/)

In [12]:
%load_ext tensorboard

In [13]:
from tensorboard import notebook
# Starten einer Tensorboard-Instanz mit Logs & gebindeten Ports
%tensorboard --port=6007 --logdir {logs_base_dir}
# Auflisten der TensorBoard-Instanzen (resourcen intesiv)
notebook.list()

Reusing TensorBoard on port 6007 (pid 8182), started 3:58:25 ago. (Use '!kill 8182' to kill it.)

Known TensorBoard instances:
  - port 6007: logdir ./logs (started 3:58:25 ago; pid 8182)


## IPyWidgets

In [14]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

## PyVirtualDisplay
*Ein python wrapper für Xvfb, Xephyr und Xvnc.*

[Weiter Informationen](https://pypi.org/project/PyVirtualDisplay/)

In [15]:
# videocode dependecies; x11-utils als workaround für 'xdpyinfo'
html_p("In case of failure in the next Code-Block consider installing python-opengl, xvfb, ffmpeg, x11-utils - Packages via apt-get.\nUse command 'sudo apt-get install -y python-opengl xvfb ffmpeg x11-utils > /dev/null' inside a terminal.",'red', True)

In [16]:
%%capture
from pyvirtualdisplay import Display as PyVDisplay

py_vdisplay = PyVDisplay(visible=0, size=(1400, 900))
py_vdisplay.start()

## OpenAI-Gym

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/OpenAI_Logo.svg/1920px-OpenAI_Logo.svg.png" width="275px"/>

*OpenAI-Gym ist ein Toolkit für das Entwickeln und Vergleichen von Reinforcement-Learning-Algorithmen.*

[Weitere Informationen](https://pypi.org/project/gym/)

In [17]:
import gym

from gym.wrappers import Monitor
from gym import logger as gymlogger

gym_loggler_lvl = 'DISABLED'

@interact
def show_loggler_lvl_param(Logger_lvl=[ 'DISABLED', 'DEBUG', 'INFO', 'WARN', 'ERROR']):
    global gym_loggler_lvl
    if Logger_lvl == 'DISABLED':
        gymlogger.set_level(50)
    elif Logger_lvl == 'DEBUG':
        gymlogger.set_level(10)
    elif Logger_lvl == 'INFO':
        gymlogger.set_level(20)
    elif Logger_lvl == 'WARN':
        gymlogger.set_level(30)
    elif Logger_lvl == 'ERROR':
        gymlogger.set_level(40)
    else:
        raise Error('Unbekanntes Logger-Level: \'{}\''.format(Logger_lvl))
    html_p('Logger-lvl auf {} gesetzt.'.format(Logger_lvl), 'green',True)

interactive(children=(Dropdown(description='Logger_lvl', options=('DISABLED', 'DEBUG', 'INFO', 'WARN', 'ERROR'…

## Gym-Super-Mario-Bros

*Eine OpenAI-Gym-Umgebung für Super Mario Bros. und Super Mario Bros. 2 (Lost Levels) auf den NES (Nitendo Entertainment System) mithilfe des nes-py emulators.*

[Weitere Informationen](https://pypi.org/project/gym-super-mario-bros/)

<table><thead><tr><th>SuperMarioBros-v0</th><th>SuperMarioBros-v1</th><th>SuperMarioBros-v2</th><th>SuperMarioBros-v3</th><th>SuperMarioBros2-v0</th><th>SuperMarioBros2-v1</th></tr></thead><tbody><tr><td><img src="https://warehouse-camo.ingress.cmh1.psfhosted.org/c4717c633d3823dda390ebc21bac34b18e7c22c3/68747470733a2f2f757365722d696d616765732e67697468756275736572636f6e74656e742e636f6d2f323138343436392f34303934383832302d33643135653563322d363833302d313165382d383164342d6563666166666565306131342e706e67" width="175"></td><td><img src="https://warehouse-camo.ingress.cmh1.psfhosted.org/e8eda56caeefcada9af67f43385ef1f48d0ac394/68747470733a2f2f757365722d696d616765732e67697468756275736572636f6e74656e742e636f6d2f323138343436392f34303934383831392d33636666366334382d363833302d313165382d383337332d3866616431363635616337322e706e67" width="175"></td><td><img src="https://warehouse-camo.ingress.cmh1.psfhosted.org/7f53e40eb716be49673cf41fb833486ab3ee104a/68747470733a2f2f757365722d696d616765732e67697468756275736572636f6e74656e742e636f6d2f323138343436392f34303934383831382d33636561303964342d363833302d313165382d386566612d3866333464386230356231312e706e67" width="175"></td><td><img src="https://warehouse-camo.ingress.cmh1.psfhosted.org/51975e7cc634efb02ed92acfb56368733b25f4d9/68747470733a2f2f757365722d696d616765732e67697468756275736572636f6e74656e742e636f6d2f323138343436392f34303934383831372d33636436363030612d363833302d313165382d386162622d3963656536613331643337372e706e67" width="175"></td><td><img src="https://warehouse-camo.ingress.cmh1.psfhosted.org/0618011a5c6cedb9dba051b8cf134ba51dd0777a/68747470733a2f2f757365722d696d616765732e67697468756275736572636f6e74656e742e636f6d2f323138343436392f34303934383832322d33643362383431322d363833302d313165382d383630622d6166333830326635333733662e706e67" width="175"></td><td><img src="https://warehouse-camo.ingress.cmh1.psfhosted.org/7c42437f4d2f447e192c088eab22739534c2d9be/68747470733a2f2f757365722d696d616765732e67697468756275736572636f6e74656e742e636f6d2f323138343436392f34303934383832312d33643264363161322d363833302d313165382d383738392d6139326537353061613961382e706e67" width="175"></td></tr></tbody></table>

In [18]:
import gym_super_mario_bros

from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import RIGHT_ONLY, SIMPLE_MOVEMENT, COMPLEX_MOVEMENT

global gym_enviorment_id
global gym_enviorment_movementset

gym_enviorment_id = 'SuperMarioBros-v0'
gym_enviorment_movementset = 'Simple'

@interact
def show_env_param(Enviorment=['SuperMarioBros-v0', 'SuperMarioBros-v1', 'SuperMarioBros-v2', 'SuperMarioBros-v3', 'SuperMarioBros2-v0', 'SuperMarioBros2-v1', 'SuperMarioBros2-v2', 'SuperMarioBros2-v3'],
                   Movementset=['Complex', 'Simple', 'Only-Right']):
    global gym_enviorment_id
    global gym_enviorment_movementset
    gym_enviorment_id = Enviorment
    gym_enviorment_movementset = Movementset
    html_p('Gym \'{}\' mit Movementset \'{}\' eingestellt.'.format(gym_enviorment_id, gym_enviorment_movementset),'green', True)

def get_actionset():
    global gym_enviorment_movementset
    if gym_enviorment_movementset == 'Simple': return SIMPLE_MOVEMENT
    elif gym_enviorment_movementset == 'Complex': return COMPLEX_MOVEMENT
    elif gym_enviorment_movementset == 'Only-Right': return RIGHT_ONLY
    else: raise Error('Unkown Movementset \"{}\".'.format(gym_enviorment_movementset))

interactive(children=(Dropdown(description='Enviorment', options=('SuperMarioBros-v0', 'SuperMarioBros-v1', 'S…

## Stable-Baselines

*Stable Baselines ist eine Sammlung von optimierten Implementation von Reinforcement-Learning-Algorithmen basierend auf OpenAi.*

[Weitere Informationen](https://pypi.org/project/stable-baselines/)

<u><b>Policyies:</b></u>
- <b>MlpPolicy</b>: Policy object that implements DQN policy, using a MLP (2 layers of 64)
- <b>LnMlpPolicy</b>: Policy object that implements DQN policy, using a MLP (2 layers of 64), with layer normalisation
- <b>CnnPolicy</b>: Policy object that implements DQN policy, using a CNN (the nature CNN)
- <b>LnCnnPolicy</b>: Policy object that implements DQN policy, using a CNN (the nature CNN), with layer normalisation

In [19]:
from stable_baselines.deepq.policies import MlpPolicy, LnMlpPolicy, CnnPolicy, LnCnnPolicy

from stable_baselines import DQN

# Trainings-Parameter

## Trainings-Epochen

In [20]:
html_p('Geben Sie eine Anzahl an zu trainierenden Epochen ein (Wertebereich: 1-∞; Default: 1).','black',True)

max_amount_of_epoches = 1

@interact
def show_epoche_picker(Epochen=str(max_amount_of_epoches)):
    global max_amount_of_epoches
    try:
        tmp = int(Epochen)
        if tmp > 0: max_amount_of_epoches = tmp
        else: max_amount_of_epoches = 1
    except ValueError:
        max_amount_of_epoches = 1
    
    html_p('Epochen-Anzahl in Höhe von {} angewandt.'.format(max_amount_of_epoches),'green',True)

interactive(children=(Text(value='1', description='Epochen'), Output()), _dom_classes=('widget-interact',))

## MP4-Aufnahmen

In [21]:
gym_enviorment_monitor = True
gym_enviorment_monitor_trigger = 0

html_p('Geben Sie an ob Aufnahmen angefertig werden werden sollen.', 'black', True)
@interact
def show_monitor_enabled_param(Aufnehmen = gym_enviorment_monitor):
    global gym_enviorment_monitor
    gym_enviorment_monitor = Aufnehmen
    if gym_enviorment_monitor: html_p('Aktiviert.','green',True)
    else: html_p('Deaktiviert.','gray',True)


html_p('Geben Sie an wann Aufnahmen angefertig werden sollen (Wertebereich: 0-∞; Default: 0).', 'black', True)

@interact
def show_monitor_trigger_param(Trigger = str(gym_enviorment_monitor_trigger)):
    global gym_enviorment_monitor_trigger
    
    try:
        tmp = int(Trigger)
        if tmp > 0: gym_enviorment_monitor_trigger = tmp
        else: gym_enviorment_monitor_trigger = 0
    except ValueError:
        gym_enviorment_monitor_trigger = 0
        
    if gym_enviorment_monitor_trigger == 0:
        html_p('Jeder Run wird aufgenommen.','green',True)
    else:
        html_p('Jeder {}. Run wird aufgenommen.'.format(gym_enviorment_monitor_trigger),'green',True)
        
        
def should_record(epoche):
    global gym_enviorment_monitor_trigger
    if gym_enviorment_monitor_trigger == 0: return True
    return gym_enviorment_monitor_trigger % epoche == 0

interactive(children=(Checkbox(value=True, description='Aufnehmen'), Output()), _dom_classes=('widget-interact…

interactive(children=(Text(value='0', description='Trigger'), Output()), _dom_classes=('widget-interact',))

# Enviorment

```
+--------------+---------------------------------------------+
|   Info-Key   |                 Beschreibung                |
+--------------+---------------------------------------------+
| coins        | Anzahl der von Mario gesammelten Münzen     |
| life         | Anzahl der Leben von Mario, {2,1,0,255}     |
| score        | Kumulativer in-game Punktestand             |
| status       | Mario's Zustand {'small','tall','fireball'} |
| flag_get     | True wenn Mario die Flagge erreicht oder aX |
| world        | Aktuele Welt {1,2,3,4,5,6,7,8}              |
| stage        | Aktuelle Etage {1,2,3,4}                    |
| time         | Aktuell übrige Zeit {400 bis 0}             |
| x_pos        | Mario's absolute X-Position                 |
| x_pos_screen | Mario's relative X-Position                 |
| y_pos        | Mario's absolute Y-Position                 |
+--------------+---------------------------------------------+
```

In [22]:
class CollectedReward:
    def __init__(self, coins, life, score, time, x_pos, y_pos):
        self.coins = coins
        self.life = life
        self.score = score
        self.time = time
        self.x_pos = x_pos
        self.y_pos = y_pos

class SuperMarioBrosEnviorment(gym.Env):
    metadata = {'render.modes': ['human']}
    
    """"""
    def __init__(self):
        super(SuperMarioBrosEnviorment, self).__init__()
        # Init Enviorment
        self._init_enviorment()
    
    """Initalizes the Enviorment"""
    def _init_enviorment(self):
        global gym_enviorment_id
        global gym_enviorment_monitor
        
        # Init
        env = gym.make(gym_enviorment_id)
        # Wrapp JoypadSpace
        env = JoypadSpace(env, get_actionset())

        # Further
        self.action_space = env.action_space
        self.observation_space = env.observation_space
        
        self._latest_reward = None
        
        # Finally
        self._env = env
    
    """Wrapped das Enviorment mit dem Monitor"""
    def wrap_monitor(self):
        global rec_base_dir
        
        # Wrapp Monitor
        if gym_enviorment_monitor: 
            self._env = Monitor( self._env,'{}/run_{}__{}'.format(rec_base_dir, datetime.now().strftime("%d-%m-%Y_%H-%M-%S"), uuid.uuid4()),
                          video_callable = lambda episode_id: should_record((episode_id - 1)), force = False )
 
    """Berrechnet den Reward"""
    def calculate_reward(self, info, gym_reward, done):
        # the reward
        reward = 0
        
        # grab vars
        coins = info['coins']
        life = info['life']
        score = info['score']
        time = info['time']
        x_pos = info['x_pos']
        y_pos = info['y_pos']
    
        if not self._latest_reward == None:
            l = self._latest_reward
            
            # Coinbonus
            reward += (coins - l.coins) * 5 # {0, 5}
            # Lifepenality
            reward += (life - l.life) * -100 # {0, -100}
            # Scorebonus
            reward += (score - l.score) * .02 # {2~20}
            # Timepenality
            timedif = (l.time - time)
            if timedif == 1: reward += -3 # {0, -3}
            reward += -1 # {0, -1}
            # Progresspenality
            if (x_pos - l.x_pos) == 0: # {0, -5}
                reward += -5
            # Progressbonus
            if not (x_pos - l.x_pos == 0) and not (y_pos - l.y_pos == 0): # {0, 2}
                reward += 2
        
        # Warning: Disable this statement else sysout will be flooded.
        if reward > 100 or reward < -100:
            print('| Coins: {:0>7d} | Life: {:0>3d} | Score: {:0>7d} | Time: {:0>3d} | X_pos: {:0>7d} | Y_pos: {:0>7d} | My_Reward: --- |'.format(l.coins,l.life,l.score,l.time,l.x_pos,l.y_pos))
            print('| Coins: {:0>7d} | Life: {:0>3d} | Score: {:0>7d} | Time: {:0>3d} | X_pos: {:0>7d} | Y_pos: {:0>7d} | My_Reward: {} |\n'.format(coins,life,score,time,x_pos,y_pos,reward))
        
        # Remeber latest reward
        self._latest_reward = CollectedReward(coins, life, score, time, x_pos, y_pos)
        
        # clip reward {-150 <-> 150}
        if reward >= 0: reward = min(reward, 150)
        else: reward = max(reward, -150)
        
        return reward
    
    """Die Step-Methode eines Enviorments"""
    def step(self, action):
        observation, reward, done, info = self._env.step(action)
        return observation, self.calculate_reward(info, reward, done), done, info
    
    """Die Reset-Methode eines Enviorments"""
    def reset(self):
        return self._env.reset()
    
    """Die Render-Methode eines Enviorments"""
    def render(self, mode='human'):
        return self._env.render(mode=mode)
    
    """Die Close-Methode eines Enviorments"""
    def close (self):
        return self._env.close()

# Hyperparameter

In [23]:
""""""
class Hyperparameter:
    
    """"""
    def __init__(self, rl_algo):
        # Load Default-Parameters
        if rl_algo == 'DQN': 
            self._default_dqn()
            self._default_dqn_learning()
        else: raise Error('Undefined Hyperparameter for RL-Algorithm \"{}\".'.format(rl_algo))
        
        # Apply Alog-Id
        self.rl_algo = rl_algo
        
        # Apply Tensorboard-Configuration
        self._apply_tensorboard()
    
    """"""
    def _apply_tensorboard(self):
        global logs_base_dir
        
        # (str) the log location for tensorboard (if None, no logging)
        self.tensorboard_log = logs_base_dir
        # (bool) enable additional logging when using tensorboard WARNING: this logging can take a lot of space quickly
        self.full_tensorboard_log = False 
    
    """"""
    def _default_dqn_learning(self):
        # (int) The total number of samples to train on
        self.total_timesteps = 1000
        # (Union[callable, [callable], BaseCallback]) function called at every steps with state of the algorithm. It takes the local and global variables. If it returns False, training is aborted. When the callback inherits from BaseCallback, you will have access to additional stages of the training (training start/end), please read the documentation for more details.
        self.callback = None
        # (int) The number of timesteps before logging.
        self.log_interval = 100
        # (str) the name of the run for tensorboard log
        self.tb_log_name = 'DQN'
        # (bool) whether or not to reset the current timestep number (used in logging)
        self.reset_num_timesteps = False
        
    """"""
    def _default_dqn(self):
        # (float) discount factor
        self.gamma = 0.95                
        # (float) learning rate for adam optimizer
        self.learning_rate = 0.005                   
        # (int) size of the replay buffer
        self.buffer_size = 50000            
        # (float) fraction of entire training period over which the exploration rate is annealed
        self.exploration_fraction = 0.2         
        # (float) final value of random action probability
        self.exploration_final_eps = 0.2         
        # (float) initial value of random action probability
        self.exploration_initial_eps = 1.0                        
        # (int) update the model every `train_freq` steps. set to None to disable printing
        self.train_freq = 50                       
        # (int) size of a batched sampled from replay buffer for training
        self.batch_size = 32                       
        # (bool) Whether to enable Double-Q learning or not.
        self.double_q = False                
        # (int) how many steps of the model to collect transitions for before learning starts
        self.learning_starts = 100      
        # (int) update the target network every `target_network_update_freq` steps.
        self.target_network_update_freq = 500            
        # (bool) if True prioritized replay buffer will be used.
        self.prioritized_replay = False
        # (float) alpha parameter for prioritized replay buffer. It determines how much prioritization is used, with alpha=0 corresponding to the uniform case.
        self.prioritized_replay_alpha = 0.6          
        # (float) initial value of beta for prioritized replay buffer      
        self.prioritized_replay_beta0 = 0.4  
        # (int) number of iterations over which beta will be annealed from initial value to 1.0. If set to None equals to max_timesteps.
        self.prioritized_replay_beta_iters = None        
        # (float) epsilon to add to the TD errors when updating priorities.
        self.prioritized_replay_eps = 1e-06                   
        # (bool) Whether or not to apply noise to the parameters of the policy.
        self.param_noise = False                  
        # (int) The number of threads for TensorFlow operations If None, the number of cpu of the current machine will be used.
        self.n_cpu_tf_sess = None                           
        # (int) the verbosity level: 0 none, 1 training information, 2 tensorflow debug
        self.verbose = 0              
        # (bool) Whether or not to build the network at the creation of the instance
        self._init_setup_model = True
        # (int) Seed for the pseudo-random generators (python, numpy, tensorflow). If None (default), use random seed. Note that if you want completely deterministicresults, you must set `n_cpu_tf_sess` to 1.
        self.seed = None                           
        # policy kw-args
        self.policy_kwargs = None

# Agent

In [24]:
""""""
class Agent:
    
    """"""
    def __init__(self, env, policy, hyp):
        # The Policy used
        self._policy = policy
        # The Enviorment used
        self._env = env
        # Setup Hyperparameters
        self._hyperparameters = hyp
        self._setup_model(hyp)
        
    """Decides which model-build-method will be triggered"""
    def _setup_model(self, hyp):
        rl_algo = hyp.rl_algo
        if rl_algo == 'DQN': self._build_dqn_model(hyp)
        #elif rl_aglo == .....
        else: raise Error('Undefined RL-Algorithm \"{}\".'.format(rl_algo))
    
    """Builds a DQN-Model with given Hyperparameters""" 
    def _build_dqn_model(self, hyp):
        self._model = DQN(
            self._policy, self._env,
            # Parameters from Hyperparameter-Obj
            gamma=hyp.gamma, learning_rate=hyp.learning_rate, buffer_size=hyp.buffer_size, exploration_fraction=hyp.exploration_fraction,
            exploration_final_eps=hyp.exploration_final_eps, exploration_initial_eps=hyp.exploration_initial_eps, train_freq=hyp.train_freq, batch_size=hyp.batch_size, double_q=hyp.double_q,
            learning_starts=hyp.learning_starts, target_network_update_freq=hyp.target_network_update_freq, prioritized_replay=hyp.prioritized_replay,
            prioritized_replay_alpha=hyp.prioritized_replay_alpha, prioritized_replay_beta0=hyp.prioritized_replay_beta0, prioritized_replay_beta_iters=hyp.prioritized_replay_beta_iters,
            prioritized_replay_eps=hyp.prioritized_replay_eps, param_noise=hyp.param_noise, n_cpu_tf_sess=hyp.n_cpu_tf_sess, verbose=hyp.verbose, tensorboard_log=hyp.tensorboard_log,
            _init_setup_model=hyp._init_setup_model, policy_kwargs=hyp.policy_kwargs, full_tensorboard_log=hyp.full_tensorboard_log, seed=hyp.seed
        )
    
    """Returns the Path where this model is saved"""
    def model_path(self):
        global model_base_dir
        return '{}/{}'.format(model_base_dir,self._hyperparameters.rl_algo)
    
    """Saves the Model"""
    def save_model(self):
        self._model.save(self.model_path())
        
    """Loads the Model"""
    def load_model(self):
        self._model.load(self.model_path())
        
    """Trains the Model"""
    def train(self, total_timesteps = None):
        hyp = self._hyperparameters
        model = self._model
        # Use default if needed
        if total_timesteps == None: total = hyp.total_timesteps
        else: total = total_timesteps
        # Delegate
        return model.learn(total_timesteps=total,callback=hyp.callback, log_interval=hyp.log_interval,
                          tb_log_name=hyp.tb_log_name,reset_num_timesteps=hyp.reset_num_timesteps)
    
    """Plays the Enviorment"""
    def play(self, times):
        # Prepare
        env = self._env
        model = self._model

        # Main loop
        for e in range(times):
        
            obs = env.reset()
            
            # In Run
            while True:
                action, _states = model.predict(obs)
                obs, reward, done, info = env.step(action)

                env.render()
                
                if done or info['flag_get']:
                    break
        # Finished
        env.close()

# Laboratory

In [ ]:
env = SuperMarioBrosEnviorment()
hyp = Hyperparameter('DQN')
agent = Agent(env, MlpPolicy, hyp)

#agent.load_model()
agent.train(50000)
#agent.save_model()

env.wrap_monitor()

agent.play(2)

/home/jan/anaconda3/lib/python3.7/site-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256
